In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from matplotlib import pyplot as plt
from e2e.deeplabv3plus import DeepLabV3plus
from utils.visual import vis_segmentation
print(tf.__version__)

2.2.0


In [4]:
IMG_SIZE = 512
NUM_CLASS = 55
AUTOTUNE = tf.data.experimental.AUTOTUNE
LEARN_RT = 1e-5
BATCH_SIZE = 7
CRF = True
INFER_ITER = 17
INFER_RATE = 0.01

In [5]:
model = DeepLabV3plus(weights=None, input_shape=(IMG_SIZE, IMG_SIZE, 3), classes=NUM_CLASS, backbone='xception',
                              OS=16, activation=None, use_crf=CRF, osi_iter=INFER_ITER, infer_rate=INFER_RATE)
rectified_adam = tfa.optimizers.RectifiedAdam(
            lr=3e-5,
            total_steps=10000,
            warmup_proportion=0.2,
            min_lr=LEARN_RT,
            epsilon=1e-08
        )
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=[tf.keras.metrics.MeanIoU(NUM_CLASS, name='mIoU'), 'sparse_categorical_accuracy'],
                      optimizer=rectified_adam)

In [14]:
weight_dir = "logs/segmentation/20210516-193213-lr_1e-05-bs_3-NoDecay-flipped_True-rotate_True-stuff512-adam-crf"
model.load_weights(weight_dir + "/006.ckpt")

In [7]:
def get_xy(record):
    img, mask, seg_ids, seg_lbs = record['image'], record['panoptic_image'], record['panoptic_objects']['id'], \
                                  record['panoptic_objects']['label']
    img = tf.image.resize(img, size=[IMG_SIZE, IMG_SIZE]) / 255.  # normalize to [0, 1] better than [-1, 1]!

    mask, seg_ids, seg_lbs = tf.cast(mask, tf.int32), tf.cast(seg_ids, tf.int32), tf.cast(seg_lbs, tf.int32)
    mask = mask[:, :, 0] + mask[:, :, 1] * 256 + mask[:, :, 2] * 256 ** 2
    # mask = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(seg_ids, seg_lbs) + 1), 0).lookup(mask)
    for i in range(len(seg_ids)):
        mask = tf.where(mask == seg_ids[i], -(seg_lbs[i] + 1), mask)
    mask = tf.abs(mask)
    mask = tf.where(tf.math.logical_and(mask > 0, mask < 81), 1, mask)  # mark all things as 1
    mask = tf.where(mask > 80, mask - 79, mask)  # offset stuff label start from 2

    # resize to 512 with Nearest neighbor interpolation
    mask = tf.image.resize(tf.expand_dims(mask, -1), size=[IMG_SIZE, IMG_SIZE], method='nearest')
    
    return img, tf.squeeze(mask, -1)

def get_data(name='coco/2017_panoptic', split='train+validation'):
    dataset = tfds.load(name, split=split, shuffle_files=True, data_dir="/data/hao/vision/")
    dataset = dataset.shuffle(buffer_size=5000, reshuffle_each_iteration=True) \
        .map(get_xy, num_parallel_calls=AUTOTUNE) \
        .batch(BATCH_SIZE, drop_remainder=False) \
        .prefetch(AUTOTUNE)
    return dataset

In [15]:
validata = get_data(name='coco/2017_panoptic', split='validation')

In [16]:
result = model.evaluate(validata, verbose=1)

715/715 [==============================] - 2807s 4s/step - loss: 186.0923 - mIoU: 0.0071 - sparse_categorical_accuracy: 0.2536


In [ ]:
samples = validata.take(3)
for k, (imgs, mask) in enumerate(samples):
    res = model.predict_on_batch(imgs)
    for img, rs in zip(imgs, res):
        vis_segmentation(tf.cast(img * 255, tf.uint8), tf.argmax(rs, -1).numpy(), ds='coco-stuff') #, save_name=f'tmp/val/internet-e77/res/{k}.jpg')